In [1]:
# imports
from netCDF4 import Dataset as NetCDFFile

# for projections manipulation
import cartopy
import cartopy.crs as ccrs

import numpy as np
import numpy.ma as ma
import pandas as pd

# plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import urllib
import urllib.request


In [2]:
geography_file = NetCDFFile(GRACEDADM_CLSM0125US_7D.A20210628.030.nc4)

SyntaxError: invalid syntax (<ipython-input-2-4691b954057e>, line 1)

# 3D plot

In [ ]:
k = 1
# unpack data into variables
lat = geography_file['lat'][::k]
lon = geography_file['lon'][::k]

gws = geography_file['gws_inst'][0, ::k, ::k]
rtzsm = geography_file['rtzsm_inst'][0, ::k, ::k]
sfsm = geography_file['sfsm_inst'][0, ::k, ::k]

# create a meshgrid for plotting
Lat, Lon = np.meshgrid(lat, lon)

# put into dataframe. plotly likes dataframes
data = pd.DataFrame(np.c_[Lat.T.flatten(), Lon.T.flatten(), gws.flatten()], columns=['Lat', 'Lon', 'gws'])

# here we are using the meshgrids (notice "Lon", not "lon")
# in order to display different levels we simply subtract a fixed value from all z coordinates of a layer
# since we do not use special map projections here (or, to be me more precise, we use the equidistant cyllindrical projection),
# we have a situation where the height of surfaces peak are too big compared to the scale of map
# so we scale down the height manually
gws_scale, rtzsm_scale, sfsm_scale = 20, 20, 20
rtzsm_shift, sfsm_shift = 20, 40

fig = go.Figure(data=[go.Surface(x=Lon, y=Lat, z=gws.filled(fill_value=np.nan).T / gws_scale,
                                 name='Ground water levels', colorscale='RdBu', showscale=False),
                      go.Surface(x=Lon, y=Lat, z=rtzsm.T.filled(fill_value=np.nan) / rtzsm_scale - rtzsm_shift,
                                 name='Root zone soil moisture', colorscale='RdBu', showscale=False),
                      go.Surface(x=Lon, y=Lat, z=sfsm.T.filled(fill_value=np.nan) / sfsm_scale - sfsm_shift,
                                 name='Surface soil moisture', colorscale='RdBu', showscale=False)])

fig.update_layout(title='USA map of ground waters', autosize=False,
                  width=500, height=500,
                  margin=dict(l=0, r=0, b=0, t=50), 
                  scene=dict(
                      #aspectmode='data',
                             xaxis=dict(title=dict(text='Lon'), 
                                        autorange='reversed'
                                       ),
                             yaxis=dict(title=dict(text='Lat'), 
                                        autorange='reversed'
                                       ),
                             zaxis=dict(title=dict(text='')),
                            ),
)
fig.show()